# 导入必要的模块

In [3]:
import numpy as np
import pandas as pd
import re
import os
import traceback
import json
import math
from bertopic import BERTopic
from itertools import product
from gensim.models.coherencemodel import CoherenceModel
from gensim.corpora import Dictionary
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import normalize
from scipy.spatial.distance import pdist
from scipy.cluster.hierarchy import linkage, fcluster
from sklearn.feature_extraction.text import CountVectorizer
from bertopic.representation import MaximalMarginalRelevance
from umap import UMAP
from hdbscan import HDBSCAN
from gensim.models.coherencemodel import CoherenceModel
import logging
from tqdm import tqdm

# 基本路径设置

In [4]:
CACHE_PATH = "/Users/hemengya/Desktop/数字贸易规则新版/第五版/embedding/merged_embedding_results.json"  
CSV_PATH = "/Users/hemengya/Desktop/数字贸易规则新版/第五版/embedding/processed_data.csv"
RESULT_CSV = "/Users/hemengya/Desktop/数字贸易规则新版/第五版/output/bertopic_docs_topics.csv"
TERMS_CSV = "/Users/hemengya/Desktop/数字贸易规则新版/第五版/output/bertopic_topic_terms.csv"
MODEL_DIR = "/Users/hemengya/Desktop/数字贸易规则新版/第五版/model/bertopic_model"
OUTPUT_DIR = "/Users/hemengya/Desktop/数字贸易规则新版/第五版/output/grid_research"
EXPORT_PATH = "/Users/hemengya/Desktop/数字贸易规则新版/第五版/output/grid_search.xlsx"

# 导入文本数据

## documents

In [5]:
data = pd.read_csv(CSV_PATH)
data = data.dropna(subset=['content_cutted']).reset_index(drop=True)
data_embedding = data.copy(deep=True)

documents = data_embedding['content_cutted'].dropna()  # 去掉 NaN
documents = documents[documents.str.strip() != ""]  # 去掉空字符串
documents = documents.tolist()

print(f"有效内容数量: {len(documents)}")

有效内容数量: 3229


## X

In [6]:
assert isinstance(documents, list), "documents 应为 list[str]，请先加载好。"

def is_numeric_vector(vec):
    if not isinstance(vec, list) or not vec:
        return False
    for x in vec:
        if not isinstance(x, (int, float)):
            return False
        fx = float(x)
        if math.isnan(fx) or math.isinf(fx):
            return False
    return True

cache = {}
invalid_entries = []  

with open(CACHE_PATH, "r", encoding="utf-8") as f:
    for line in f:
        try:
            # 逐行解析每个 JSON 对象
            obj = json.loads(line.strip())  # 读取并去掉空格
            # 假设每行的 JSON 对象包含 key 和对应的向量
            key = str(obj["key"])  # 获取 key
            embedding = obj.get("embedding")
            cache[key] = embedding
        except json.JSONDecodeError as e:
            invalid_entries.append({"key": line.strip(), "reason": f"无法解析行，错误: {e}"})

valid_indices = []
texts = []
embeds = []

N = len(documents)
for i in range(N):
    key = str(i + 1)  # 将索引转为字符串
    vec = cache.get(key)  # 从缓存中获取该 key 对应的向量
    if is_numeric_vector(vec):  # 如果该向量有效
        valid_indices.append(i)
        texts.append(documents[i])  # 保留文本
        embeds.append(vec)  # 保留嵌入向量
    else:
        invalid_entries.append({"key": key, "reason": "无效向量"})

# 输出有效条目数量
print(f"可用条目：{len(valid_indices)}/{N}（其余因缺失/无效向量被过滤）")

X = np.array(embeds, dtype=float)

# 输出不可用条目的详细信息
if invalid_entries:
    print("不可用条目详细信息：")
    for entry in invalid_entries:
        print(f"Key: {entry['key']}, 原因: {entry['reason']}")
else:
    print("没有发现不可用条目。")


可用条目：3229/3229（其余因缺失/无效向量被过滤）
没有发现不可用条目。


## 日志

In [7]:
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
# 抑制 BERTopic 和 HDBSCAN 的 INFO 级别日志
logging.getLogger('bertopic').setLevel(logging.WARNING)
logging.getLogger('hdbscan').setLevel(logging.WARNING)

### 定义参数组合

In [25]:
param_grid = {
    'n_neighbors': [10, 15, 20, 25],
    'n_components': [5, 10, 15, 20],
    'min_cluster_size': [2, 3, 4, 5],
}


def generate_param_combinations(param_dict):
    import itertools
    keys = param_dict.keys()
    values = param_dict.values()
    combinations = [dict(zip(keys, p)) for p in itertools.product(*values)]
    return combinations

param_combinations = generate_param_combinations(param_grid)
logging.info(f"将运行 {len(param_combinations)} 种参数组合。")

2025-09-25 20:10:41,172 - root - INFO - 将运行 64 种参数组合。


### 辅助函数

In [26]:
def auto_threshold(merge_d):
    if merge_d.size < 2:
        return float(merge_d[-1] if merge_d.size else 0.0)
    jumps = (merge_d[1:] - merge_d[:-1]) / np.maximum(merge_d[:-1], 1e-9)
    k = int(np.argmax(jumps))
    th = float((merge_d[k] + merge_d[k+1]) / 2.0)
    lo, hi = np.percentile(merge_d, [50, 95])
    return float(np.clip(th, lo, hi))

def is_valid_token(w: str) -> bool:
    if not isinstance(w, str) or not w: return False
    if re.search(r"\d", w): return False
    if re.fullmatch(r"[\W_]+", w): return False
    return len(w) > 2

def cluster_metrics(X, labels, sample_cap=6000, seed=42):
    rng = np.random.default_rng(seed)
    labels = np.asarray(labels, dtype=int)
    outlier_rate = float(np.mean(labels == -1))
    
    def cosine(u, v):
        du = np.linalg.norm(u); dv = np.linalg.norm(v)
        if du == 0 or dv == 0: return 0.0
        return float(np.dot(u, v) / (du * dv))
    sims = []
    sizes = []
    for c in np.unique(labels):
        if c < 0: continue
        idx = np.where(labels == c)[0]
        sizes.append(len(idx))
        if len(idx) >= 2:
            max_pairs = min(400, len(idx)*(len(idx)-1)//2)
            if max_pairs > 0:
                for _ in range(max_pairs):
                    try:
                        a, b = rng.choice(idx, size=2, replace=False)
                        sims.append(cosine(X[a], X[b]))
                    except ValueError:
                        break
    intra_mean = float(np.mean(sims)) if sims else float('nan')
    sil = float('nan')
    keep = np.where(labels >= 0)[0]
    if keep.size >= 20:
        if keep.size > sample_cap:
            keep = rng.choice(keep, size=sample_cap, replace=False)
        sil = silhouette_score(X[keep], labels[keep], metric="cosine")
    n_clusters = int(np.sum(np.unique(labels) >= 0))
    avg_size = float(np.mean(sizes)) if sizes else 0.0
    return intra_mean, sil, outlier_rate, n_clusters, avg_size

def calculate_coherence_robust(topic_model, topics, texts, dictionary):
    valid_topic_words = []
    topic_words_dict = topic_model.get_topics()
    for topic_id in topic_words_dict.keys():
        if topic_id == -1:
            continue
        raw_words = topic_model.get_topic(topic_id)
        topic_words_for_gensim = []
        for word, _ in raw_words:
            tokens = re.findall(r'\b[a-zA-Z]+\b', word.lower())
            for token in tokens:
                if token in dictionary.token2id:
                    topic_words_for_gensim.append(token)
        if topic_words_for_gensim:
            valid_topic_words.append(topic_words_for_gensim)
    if not valid_topic_words:
        logging.warning("没有生成任何有效主题，C_V 一致性为 0。")
        return 0.0
    try:
        coherence_model = CoherenceModel(
            topics=valid_topic_words,
            texts=texts,
            dictionary=dictionary,
            coherence='c_v'
        )
        return coherence_model.get_coherence()
    except Exception as e:
        logging.error(f"Gensim 计算 C_V 失败，错误信息：{e}")
        return 0.0
    

### 辅助一致性计算

In [27]:
tokenized_docs = []
for doc in documents:
    words = re.findall(r'\b[a-zA-Z]+\b', doc.lower())
    tokenized_docs.append(words)
dictionary = Dictionary(tokenized_docs)
logging.info(f"Gensim 词典创建完成，包含 {len(dictionary)} 个词汇。")
valid_idx = np.arange(len(documents))
    
param_combinations = [dict(zip(param_grid.keys(), v)) for v in product(*param_grid.values())]

2025-09-25 20:10:45,725 - gensim.corpora.dictionary - INFO - adding document #0 to Dictionary<0 unique tokens: []>
2025-09-25 20:10:45,852 - gensim.corpora.dictionary - INFO - built Dictionary<8824 unique tokens: ['add', 'administrative', 'aspect', 'assess', 'basis']...> from 3229 documents (total 358966 corpus positions)
2025-09-25 20:10:45,853 - gensim.utils - INFO - Dictionary lifecycle event {'msg': "built Dictionary<8824 unique tokens: ['add', 'administrative', 'aspect', 'assess', 'basis']...> from 3229 documents (total 358966 corpus positions)", 'datetime': '2025-09-25T20:10:45.853074', 'gensim': '4.3.3', 'python': '3.10.16 (main, Dec 11 2024, 10:22:29) [Clang 14.0.6 ]', 'platform': 'macOS-15.0.1-arm64-arm-64bit', 'event': 'created'}
2025-09-25 20:10:45,853 - root - INFO - Gensim 词典创建完成，包含 8824 个词汇。


In [28]:
results = []

SIM_THRESHOLD = 0.9 
ABSORB_THRESHOLD = 0.7
TOP_K = 30

for i, params in enumerate(tqdm(param_combinations, desc="正在进行参数组合训练")):
    try:
        logging.info(f"\n--- 开始第 {i+1}/{len(param_combinations)} 次训练，参数: {params} ---")
        
        # 1. 配置 BERTopic 模型
        umap_model = UMAP(
            n_neighbors=params['n_neighbors'],
            n_components=params['n_components'],
            min_dist=0.0,
            metric="cosine",
            random_state=42
        )
        hdbscan_model = HDBSCAN(
            min_cluster_size=params['min_cluster_size'],
            metric="euclidean",
            cluster_selection_method="leaf",
            prediction_data=True
        )
        vectorizer_model = CountVectorizer(
            ngram_range=(1, 2),
            min_df=2,
            max_df=0.9
        )
        representation_model = MaximalMarginalRelevance(diversity=0.0)
        
        topic_model = BERTopic(
            embedding_model=None,
            umap_model=umap_model,
            hdbscan_model=hdbscan_model,
            vectorizer_model=vectorizer_model,
            representation_model=representation_model,
            nr_topics="auto",
            calculate_probabilities=True,
            verbose=False
        )
        
        # 2. 训练 BERTopic 模型
        topics, probs = topic_model.fit_transform(documents, embeddings=X)
        topics = np.array(topics, dtype=int)
        
        # 3. 主题合并流程
        mask_in = topics >= 0
        topic_ids = np.unique(topics[mask_in])
        centroids, tid_list = [], []
        for tid in topic_ids:
            doc_idx = np.where(topics == tid)[0]
            if len(doc_idx) > 0:
                centroids.append(X[doc_idx].mean(axis=0))
                tid_list.append(int(tid))
        
        if not centroids:
            raise ValueError("No valid topics found for merging.")
            
        centroids = normalize(np.vstack(centroids))
        dist = pdist(centroids, metric="cosine")
        Z = linkage(dist, method="average")
        merge_d = Z[:, 2]
        
        # 相似度阈值来合并主题
        if SIM_THRESHOLD is None:
            distance_threshold = auto_threshold(merge_d)
        else:
            distance_threshold = float(1.0 - SIM_THRESHOLD)
        
        super_labels_topics = fcluster(Z, t=distance_threshold, criterion="distance")
        tid2super = {tid: sup for tid, sup in zip(tid_list, super_labels_topics)}
        topics_super = np.array([tid2super.get(tid, -1) for tid in topics])
        n_super = len(np.unique(topics_super))
        
        # 4. 离群点吸收
        super_ids = sorted(np.unique(super_labels_topics))
        super_centroids = []
        for sup in super_ids:
            member_tids = [tid for tid, s in tid2super.items() if s == sup]
            member_doc_idx = np.where(np.isin(topics, member_tids))[0]
            if len(member_doc_idx) > 0:
                super_centroids.append(X[member_doc_idx].mean(axis=0))
        
        if super_centroids:
            super_centroids = normalize(np.vstack(super_centroids))
            def cosine_sim_rows(A, b):
                b = b / (np.linalg.norm(b) + 1e-12)
                return A.dot(b)

            # 处理遗漏数据
            out_idx = np.where(topics_super == -1)[0]
            if out_idx.size:
                sims = np.vstack([cosine_sim_rows(super_centroids, X[i]) for i in out_idx])
                best_sup = np.argmax(sims, axis=1)
                best_val = sims[np.arange(sims.shape[0]), best_sup]
                absorb_mask = best_val >= ABSORB_THRESHOLD
                topics_super[out_idx[absorb_mask]] = np.array([super_ids[j] for j in best_sup[absorb_mask]], dtype=int)
                logging.info(f"吸收离群点：候选 {out_idx.size} 篇 → 吸收 {absorb_mask.sum()} 篇（阈值={ABSORB_THRESHOLD:.2f}）")

        # 5. 生成合并后超主题的可读标签
        topic_dict = topic_model.get_topics()
        df_topic_info = topic_model.get_topic_info().set_index("Topic")
        super_id_to_words = {}
        for sup in super_ids:
            member_tids = [tid for tid, s in tid2super.items() if s == sup]
            words_pool = []
            for tid in member_tids:
                pairs = topic_dict.get(int(tid), [])
                words_pool.extend([w for (w, s) in pairs if is_valid_token(w)])
            if words_pool:
                vc = pd.Series(words_pool).value_counts()
                super_id_to_words[sup] = ", ".join(vc.index[:TOP_K].tolist())
            else:
                names = []
                for tid in member_tids:
                    if tid in df_topic_info.index:
                        nm = str(df_topic_info.loc[tid, "Name"])
                        names.append(nm.split("_", 1)[-1] if "_" in nm else nm)
                super_id_to_words[sup] = " | ".join(names[:TOP_K]) if names else f"SuperTopic {sup}"

        # 6. 计算指标和 C_V 一致性
        m_before = cluster_metrics(X, topics)
        m_after = cluster_metrics(X, topics_super)
        cv_coherence = calculate_coherence_robust(topic_model, topics, tokenized_docs, dictionary)

        # 7. 将结果写回原始行并导出
        current_data = pd.read_csv(CSV_PATH)
        labels_full = np.full(len(documents), -1, dtype=int)
        labels_full[valid_idx] = topics_super
        label_text = pd.Series(labels_full).map(super_id_to_words).fillna("<No keywords>")

        current_data["hc_topic_label"] = labels_full
        current_data["hc_topic_label_text"] = label_text

        # 动态生成文件名
        file_name = f"result_n_comp_{params['n_components']}_msize_{params['min_cluster_size']}_neig_{params['n_neighbors']}.xlsx"
        export_path = os.path.join(OUTPUT_DIR, file_name)

        try:
            current_data.to_excel(export_path, index=False)
            logging.info(f"已导出文档和超主题结果：{export_path}")
        except Exception as e:
            logging.error(f"导出失败，错误：{e}")

        # 8. 存储结果到汇总列表
        result = {
            'n_neighbors': params['n_neighbors'],
            'n_components': params['n_components'],
            'min_cluster_size': params['min_cluster_size'],
            'num_original_topics': len(topic_ids),
            'num_merged_topics': n_super,
            'cv_coherence': cv_coherence,
            'intra_mean_before': m_before[0],
            'silhouette_before': m_before[1],
            'outlier_rate_before': m_before[2],
            'intra_mean_after': m_after[0],
            'silhouette_after': m_after[1],
            'outlier_rate_after': m_after[2]
        }
        results.append(result)
        
        logging.info(f"本次训练完成。原主题数: {result['num_original_topics']}，合并后超主题数: {result['num_merged_topics']}，C_V 一致性: {result['cv_coherence']:.4f}")
        logging.info(f"指标：轮廓系数 (合并前/后): {result['silhouette_before']:.4f}/{result['silhouette_after']:.4f}, 离群率: {result['outlier_rate_after']:.4f}")

    except Exception as e:
        logging.error(f"训练失败，参数组合 {params}。错误信息：{e}")
        continue


正在进行参数组合训练:   0%|          | 0/64 [00:00<?, ?it/s]2025-09-25 20:10:52,667 - root - INFO - 
--- 开始第 1/64 次训练，参数: {'n_neighbors': 10, 'n_components': 5, 'min_cluster_size': 2} ---
2025-09-25 20:11:32,836 - root - INFO - 吸收离群点：候选 686 篇 → 吸收 670 篇（阈值=0.70）
2025-09-25 20:11:34,493 - gensim.topic_coherence.probability_estimation - INFO - using ParallelWordOccurrenceAccumulator<processes=7, batch_size=64> to estimate probabilities from sliding windows
2025-09-25 20:11:38,465 - gensim.topic_coherence.text_analysis - INFO - 1 batches submitted to accumulate stats from 64 documents (2974 virtual)
2025-09-25 20:11:38,468 - gensim.topic_coherence.text_analysis - INFO - 2 batches submitted to accumulate stats from 128 documents (4038 virtual)
2025-09-25 20:11:38,474 - gensim.topic_coherence.text_analysis - INFO - 8 batches submitted to accumulate stats from 512 documents (-2575 virtual)
2025-09-25 20:11:39,397 - gensim.topic_coherence.text_analysis - INFO - 22 batches submitted to accumulate stats 

### 导出训练结果

In [29]:
results_df = pd.DataFrame(results)
print("\n--- 所有训练结果 ---")
print(results_df)

best_result = results_df.sort_values(by='cv_coherence', ascending=False).iloc[0]
print("\n--- C_V 一致性最佳参数组合 ---")
print(best_result)

results_df.to_excel(EXPORT_PATH, index=False)
logging.info(f"结果已导出至 {EXPORT_PATH}")

2025-09-25 20:58:46,100 - root - INFO - 结果已导出至 /Users/hemengya/Desktop/数字贸易规则新版/第五版/output/grid_search.xlsx



--- 所有训练结果 ---
    n_neighbors  n_components  min_cluster_size  num_original_topics  \
0            10             5                 2                  197   
1            10             5                 3                  153   
2            10             5                 4                  138   
3            10             5                 5                  124   
4            10            10                 2                  190   
..          ...           ...               ...                  ...   
59           25            15                 5                   98   
60           25            20                 2                  165   
61           25            20                 3                  138   
62           25            20                 4                  121   
63           25            20                 5                  104   

    num_merged_topics  cv_coherence  intra_mean_before  silhouette_before  \
0                 179      0.607182       